In [3]:
from google.colab import drive 
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [4]:
!pip install pydot --quiet
!pip install gensim==3.8.3 --quiet
!pip install tensorflow-datasets --quiet
!pip install -U tensorflow-text==2.8.2 --quiet
!pip install transformers --quiet

     |████████████████████████████████| 24.2 MB 1.2 MB/s 
     |████████████████████████████████| 4.9 MB 14.0 MB/s 
     |████████████████████████████████| 498.0 MB 12 kB/s 
     |████████████████████████████████| 1.4 MB 62.8 MB/s 
     |████████████████████████████████| 5.8 MB 41.6 MB/s 
     |████████████████████████████████| 462 kB 46.0 MB/s 
     |████████████████████████████████| 5.8 MB 13.5 MB/s 
     |████████████████████████████████| 7.6 MB 58.9 MB/s 
     |████████████████████████████████| 182 kB 70.5 MB/s 


In [5]:
import pandas as pd
import numpy as np
import sys
import os
import re

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.layers import Embedding, Input, Dense, Lambda
from tensorflow.keras.models import Model
import tensorflow.keras.backend as K
import tensorflow_datasets as tfds
import tensorflow_text as tf_text

from transformers import BertTokenizer, TFBertModel


import sklearn as sk

import nltk
from nltk.data import find

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


#This continues to work with gensim 3.8.3.  It doesn't yet work with 4.x.  
#Make sure your pip install command specifies gensim==3.8.3
import gensim
from gensim.models import word2vec, FastText
from gensim.utils import tokenize

In [6]:
os.chdir('/content/gdrive/MyDrive/Windows/')

In [7]:
df=pd.read_csv('Windows_log_labeled.csv')

In [8]:
df.head()

,Unnamed: 0,LineId,Date,Time,Type,Admin,Content,EventId,EventTemplate,Label
0,0,1,9/27/2016,4:30:30,Info,CBS,Starting TrustedInstaller initialization.,4351fc0e,<*> TrustedInstaller <*>,0
1,1,2,9/28/2016,4:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,b642288b,Loaded Servicing Stack v<*> with Core: C:\Wind...,0
2,2,3,9/28/2016,4:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
3,3,4,9/28/2016,4:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
4,4,5,9/28/2016,4:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0


In [9]:
nltk.download('word2vec_sample')

word2vec_sample = str(find('models/word2vec_sample/pruned.word2vec.txt'))

model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_sample, binary=False)

[nltk_data] Downloading package word2vec_sample to /root/nltk_data...
[nltk_data]   Unzipping models/word2vec_sample.zip.


In [10]:
whole_data_emb = df["Content"].values.tolist()
whole_data_label=df["Label"].values.tolist()

In [11]:
len(whole_data_emb)

79966

In [12]:
# from sklearn.model_selection import train_test_split
# X_train, X_val, y_train, y_val = train_test_split(whole_data_emb, whole_data_label, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2

In [13]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10



X_train, X_test, y_train, y_test = train_test_split(whole_data_emb, whole_data_label, test_size=test_ratio, random_state=1)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio))

print("Train Size:",len(X_train))
print("Test Size:",len(X_test))
print("Valid Size:",len(X_valid))

Train Size: 63972
Test Size: 7997
Valid Size: 7997


In [14]:
#@title Embedding Matrix Creation

EMBEDDING_DIM = 300    # we know... it's 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = np.zeros((len(model.vocab.keys()) + 1, EMBEDDING_DIM))       
vocab_dict = {}

# build the embedding matrix and the word-to-id map:
for i, word in enumerate(model.vocab.keys()):
    embedding_vector = model[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
        vocab_dict[word] = i

# we can use the last index at the end of the vocab for unknown tokens
vocab_dict['[UNK]'] = len(vocab_dict)

In [15]:
SEQUENCE_LENGTH = 100

In [16]:
def docs_to_vocab_ids(tokenized_texts_list):
    """
    converting a list of strings to a list of lists of word ids
    """
    texts_vocab_ids = []
    text_labels = []
    valid_example_list = []
    for i, token_list in enumerate(tokenized_texts_list):

        # Get the vocab id for each token in this doc ([UNK] if not in vocab)
        vocab_ids = []
        for token in list(token_list.numpy()):
            decoded = token.decode('utf-8', errors='ignore')
            if decoded in vocab_dict:
                vocab_ids.append(vocab_dict[decoded])
            else:
                vocab_ids.append(vocab_dict['[UNK]'])
            
        # Truncate text to max length, add padding up to max length
        vocab_ids = vocab_ids[:SEQUENCE_LENGTH]
        n_padding = (SEQUENCE_LENGTH - len(vocab_ids))
        # For simplicity in this model, we'll just pad with unknown tokens
        vocab_ids += [vocab_dict['[UNK]']] * n_padding
        valid_example_list.append(i)
        # Add this example to the list of converted docs
        texts_vocab_ids.append(vocab_ids)
            
        if i % 5000 == 0:
            print('Examples processed: ', i)

    print('Total examples: ', i)
    return (np.array(texts_vocab_ids), valid_example_list)

In [17]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [18]:
X_train.shape

(63972,)

In [19]:
X_test.shape

(7997,)

In [20]:
tokenizer = tf_text.WhitespaceTokenizer()
train_tokens = tokenizer.tokenize(X_train[:500000])
test_tokens= tokenizer.tokenize(X_test[:150000])                          

In [21]:
ytrain=np.array(y_train[:500000])
ytest=np.array(y_test[:150000])

In [22]:
train_tokens[0:3]

<tf.RaggedTensor [[b'Applicability(ComponentAnalyzerEvaluateSelfUpdate):', b'Component:',
  b'wow64_microsoft-windows-appid_31bf3856ad364e35_6.1.7601.23126_none_c0475672164bc82a,',
  b'elevate:', b'2,', b'applicable(true/false):', b'0']                                   ,
 [b'Appl:', b'Selfupdate,', b'Component:',
  b'amd64_microsoft-windows-c..ityclient.resources_31bf3856ad364e35_0.0.0.0_zh-cn_443e5289877e40b1',
  b'(6.1.7601.23126),', b'elevation:2,', b'lower', b'version', b'revision',
  b'holder:', b'6.1.7601.18833']                                                                    ,
 [b'Warning:', b'Unrecognized', b'packageExtended', b'attribute.']]>

In [23]:
test_tokens[0:3]

<tf.RaggedTensor [[b'Appl:', b'Selfupdate,', b'Component:',
  b'x86_microsoft-windows-m..ditevtlog.resources_31bf3856ad364e35_0.0.0.0_es-es_7daaf684a22d94a8',
  b'(6.1.7601.23126),', b'elevation:2,', b'lower', b'version', b'revision',
  b'holder:', b'0.0.0.0']                                                                         ,
 [b'Appl:', b'detect', b'Parent,', b'Package:',
  b'Package_for_KB3060716~31bf3856ad364e35~amd64~~6.1.1.0,', b'Parent:',
  b'Microsoft-Windows-WinPE-LanguagePack-Package~31bf3856ad364e35~amd64~bg-BG~6.1.7601.17514,',
  b'Disposition', b'=', b'Detect,', b'VersionComp:', b'EQ,',
  b'ServiceComp:', b'EQ,', b'BuildComp:', b'EQ,', b'DistributionComp:',
  b'GE,', b'RevisionComp:', b'GE,', b'Exist:', b'present']                                     ,
 [b'Appl:', b'Selfupdate,', b'Component:',
  b'amd64_netfx35linq-system.web.extensions_31bf3856ad364e35_0.0.0.0_none_4e723abaebb210fc',
  b'(6.1.7601.21884),', b'elevation:2,', b'lower', b'version', b'revision',
  b'ho

In [24]:
train_input_ids, train_valid_example_list = docs_to_vocab_ids(train_tokens)
test_input_ids, test_valid_example_list = docs_to_vocab_ids(test_tokens)

Examples processed:  0
Examples processed:  5000
Examples processed:  10000
Examples processed:  15000
Examples processed:  20000
Examples processed:  25000
Examples processed:  30000
Examples processed:  35000
Examples processed:  40000
Examples processed:  45000
Examples processed:  50000
Examples processed:  55000
Examples processed:  60000
Total examples:  63971
Examples processed:  0
Examples processed:  5000
Total examples:  7996


In [25]:
train_input_ids[:2]

array([[43981, 43981, 43981, 43981, 43981, 43981,  6171, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981],
       [43981, 43981, 43981, 43981, 43981, 43981, 21120,  9626,  2072,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 4398

In [26]:
test_input_ids[:2]

array([[43981, 43981, 43981, 43981, 43981, 43981, 21120,  9626,  2072,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981],
       [43981, 20307, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981, 43981,
        43981, 43981, 43981, 32772, 43981, 43981, 43981, 4398

In [27]:
from sklearn.metrics import confusion_matrix
def conf_matrix(model,xtest1,ytest1):
  y_pred = model.predict(xtest1)
  
  y_pred = np.argmax(y_pred, axis=1)
  conf_mat = confusion_matrix(ytest1, y_pred)
  TP=conf_mat[0][0]
  FP=conf_mat[0][1]
  FN=conf_mat[1][0]
  TN=conf_mat[1][1]
  Precision = TP*100/(TP+FP)
  print('Precision:',Precision) 
  recall=TP*100/(TP+FN)
  print('Recall:',recall)
  F1_score=(2*TP*100)/(2*TP+FP+FN)
  print('F1 Score:',F1_score)

  return recall, F1_score, conf_mat

In [28]:
def create_dan_model(retrain_embeddings=False, 
                     max_sequence_length=SEQUENCE_LENGTH,
                     hidden_dim=100,
                     dropout=0.3,
                     embedding_initializer='word2vec', 
                     learning_rate=0.001):
  """
  Construct the DAN model including the compilation and return it. Parametrize it using the arguments.
  :param retrain_embeddings: boolean, indicating whether  the word embeddings are trainable
  :param hidden_dim: dimension of the hidden layer
  :param dropout: dropout applied to the hidden layer

  :returns: the compiled model
  """

  if embedding_initializer == 'word2vec':
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix)
  else:
    embeddings_initializer='uniform'

  dan_input_layer = tf.keras.layers.Input(shape=(max_sequence_length,), dtype='int64')
  
  dan_embedding_layer = Embedding(embedding_matrix.shape[0],
                                    embedding_matrix.shape[1],
                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                    input_length=SEQUENCE_LENGTH,
                                    trainable=retrain_embeddings)
  
  dan_embeddings = dan_embedding_layer(dan_input_layer)
    
  dan_avg_embeddings = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1),name='averaging')(dan_embeddings)
  
  last_hidden_output = tf.keras.layers.Dense(100, activation='relu',
                                                   name='dan_hidden_%d' % i)(dan_avg_embeddings)

  last_hidden_output = tf.keras.layers.Dropout(dropout)(last_hidden_output)

  dan_classification = tf.keras.layers.Dense(1,
                                               activation='sigmoid',
                                               name='dan_classification')(last_hidden_output)

    
  dan_model1 = tf.keras.models.Model(inputs=dan_input_layer, outputs=dan_classification)
    
    
  dan_model1.compile(loss='binary_crossentropy',
                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                         beta_1=0.9,
                                                         beta_2=0.999,
                                                         epsilon=1e-07,
                                                         amsgrad=False,
                                                         name='Adam'),
                      metrics='accuracy')
  
  print(dan_model1.summary())
  return dan_model1


In [29]:
dan_model1 = create_dan_model()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 300)          13194600  
                                                                 
 averaging (Lambda)          (None, 300)               0         
                                                                 
 dan_hidden_43980 (Dense)    (None, 100)               30100     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dan_classification (Dense)  (None, 1)                 101       
                                                                 
Total params: 13,224,801
Trainable params: 30,201
Non-trainab

In [30]:

history_noshuffle = dan_model1.fit(train_input_ids,
                            ytrain,
                            validation_data=(test_input_ids,ytest),
                            batch_size=32,
                            epochs=5,
                            shuffle=True
                            )

Epoch 1/5
2000/2000 [==============================] - 9s 3ms/step - loss: 0.1376 - accuracy: 0.9520 - val_loss: 0.0512 - val_accuracy: 0.9974
Epoch 2/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0384 - accuracy: 0.9966 - val_loss: 0.0231 - val_accuracy: 0.9972
Epoch 3/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0168 - accuracy: 0.9979 - val_loss: 0.0089 - val_accuracy: 1.0000
Epoch 4/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0077 - accuracy: 0.9996 - val_loss: 0.0040 - val_accuracy: 1.0000
Epoch 5/5
2000/2000 [==============================] - 6s 3ms/step - loss: 0.0044 - accuracy: 0.9997 - val_loss: 0.0020 - val_accuracy: 1.0000


In [31]:
conf_matrix(dan_model1,test_input_ids,ytest)

Precision: 100.0
Recall: 93.08490684006503
F1 Score: 96.41862573667508


(93.08490684006503, 96.41862573667508, array([[7444,    0],
        [ 553,    0]]))

In [32]:
def create_wan_model(retrain_embeddings=False, 
                     max_sequence_length=SEQUENCE_LENGTH,
                     hidden_dim=100,
                     dropout=0.3,
                     learning_rate=0.001):
  """
  Construct the WAN model including the compilation and return it. Parametrize it using the arguments.
  :param retrain_embeddings: boolean, indicating whether the word embeddings are trainable
  :param hidden_dim: dimension of the hidden layer
  :param dropout: dropout applied to the hidden layer

  :returns: the compiled model
  """
  ### YOUR CODE HERE

  #if embedding_initializer == 'word2vec':
      #embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix)
  #else:
      #embeddings_initializer='uniform'

  wan_input_layer = tf.keras.layers.Input(shape=(max_sequence_length,), dtype='int64') #fake input layer
 

  wan_embedding_layer = Embedding(embedding_matrix.shape[0],
                                    embedding_matrix.shape[1],
                                    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                                    input_length=SEQUENCE_LENGTH,
                                    trainable=retrain_embeddings)
  
  
  wan_embeddings = wan_embedding_layer(wan_input_layer)
  
  #query
  wan_query_layer = tf.keras.layers.Dense(embedding_matrix.shape[1]) #query
  
  wan_one_vector = tf.Variable(tf.ones((1, 1, 1)))
  
  wan_batch_of_ones = tf.tile(wan_one_vector, (tf.shape(wan_input_layer)[0], 1, 1)) 

  wan_query_vector = wan_query_layer(wan_batch_of_ones)

    
  wan_avg_embeddings = tf.keras.layers.Lambda(lambda x: K.mean(x, axis=1),name='averaging')(wan_embeddings)

  wan_attention_output,wan_attention_weights= tf.keras.layers.Attention()([wan_query_vector,wan_embeddings],return_attention_scores=True)

  
  wan_attention_output = tf.keras.layers.Reshape((wan_attention_output.shape[-1],))(wan_attention_output)


  
  last_hidden_output = tf.keras.layers.Dense(100, activation='relu',
                                                   name='wan_hidden_%d' % i)(wan_attention_output)

  last_hidden_output = tf.keras.layers.Dropout(dropout)(last_hidden_output)

  wan_classification = tf.keras.layers.Dense(1,
                                               activation='sigmoid',
                                               name='classification')(last_hidden_output)

    
  wan_model1 = tf.keras.models.Model(inputs=wan_input_layer, outputs=[wan_classification,wan_attention_weights])
    
    
  wan_model1.compile(loss=['binary_crossentropy',None],
                      optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,
                                                         beta_1=0.9,
                                                         beta_2=0.999,
                                                         epsilon=1e-07,
                                                         amsgrad=False,
                                                         name='Adam'),
                      metrics='accuracy')
  
  print(wan_model1.summary())
  
  return wan_model1


In [33]:

wan_model = create_wan_model()
history_wan = wan_model.fit(train_input_ids,
                            ytrain,
                            validation_data=(test_input_ids,ytest),
                            batch_size=32,
                            epochs=10,
                            shuffle=True
                            )
# use wan_history = ... below

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 tf.compat.v1.shape (TFOpLambda  (2,)                0           ['input_2[0][0]']                
 )                                                                                                
                                                                                                  
 tf.__operators__.getitem (Slic  ()                  0           ['tf.compat.v1.shape[0][0]']     
 ingOpLambda)                                                                                     
                                                                                            

In [34]:
def conf_matrix1(model,xtest1,ytest1):
  y_pred = model.predict(xtest1)
  y_pred=y_pred[0:len(X_valid)][0]
  y_pred=np.array(y_pred)
  y_pred = np.argmax(y_pred, axis=1)
  conf_mat = confusion_matrix(ytest1, y_pred)
  TP=conf_mat[0][0]
  FP=conf_mat[0][1]
  FN=conf_mat[1][0]
  TN=conf_mat[1][1]
  Precision = TP*100/(TP+FP)
  print('Precision:',Precision) 
  recall=TP*100/(TP+FN)
  print('Recall:',recall)
  F1_score=(2*TP*100)/(2*TP+FP+FN)
  print('F1 Score:',F1_score)

  return recall, F1_score, conf_mat

In [35]:
conf_matrix1(wan_model,test_input_ids,ytest)

Precision: 100.0
Recall: 93.08490684006503
F1 Score: 96.41862573667508


(93.08490684006503, 96.41862573667508, array([[7444,    0],
        [ 553,    0]]))

In [36]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [37]:
bert_model = TFBertModel.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [38]:
df1=pd.read_csv('Windows_log_labeled.csv')

In [39]:
df1.head()

,Unnamed: 0,LineId,Date,Time,Type,Admin,Content,EventId,EventTemplate,Label
0,0,1,9/27/2016,4:30:30,Info,CBS,Starting TrustedInstaller initialization.,4351fc0e,<*> TrustedInstaller <*>,0
1,1,2,9/28/2016,4:30:30,Info,CBS,Loaded Servicing Stack v6.1.7601.23505 with Co...,b642288b,Loaded Servicing Stack v<*> with Core: C:\Wind...,0
2,2,3,9/28/2016,4:30:31,Info,CSI,00000001@2016/9/27:20:30:31.455 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
3,3,4,9/28/2016,4:30:31,Info,CSI,00000002@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0
4,4,5,9/28/2016,4:30:31,Info,CSI,00000003@2016/9/27:20:30:31.458 WcpInitialize ...,a742c417,<*>@<*>/<*>/<*>:<*>:<*>:<*>.<*> WcpInitialize ...,0


In [40]:
whole_data_emb = df1["Content"].values.tolist()
whole_data_label=df1["Label"].values.tolist()

In [41]:
train_ratio = 0.80
test_ratio = 0.10
validation_ratio = 0.10



X_train, X_test, y_train, y_test = train_test_split(whole_data_emb, whole_data_label, test_size=test_ratio, random_state=1)

X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=validation_ratio/(train_ratio+test_ratio))

print("Train Size:",len(X_train))
print("Test Size:",len(X_test))
print("Valid Size:",len(X_valid))

Train Size: 63972
Test Size: 7997
Valid Size: 7997


In [42]:

# num_train_examples = len(X_train)*.1
# num_test_examples = len(X_test)*.1
# num_val_examples=len(X_valid)*.1
# max_length = 50

In [43]:
# #num_train_examples = 250000
# num_train_examples = 50000
# num_test_examples = 20000
# num_val_examples=20000
max_length = 50


# x_train = bert_tokenizer(X_train[:num_train_examples], 
#               max_length=max_length,
#               truncation=True,
#               padding='max_length', 
#               return_tensors='tf')

# x_test = bert_tokenizer(X_test[:num_test_examples], 
#               max_length=max_length,
#               truncation=True,
#               padding='max_length', 
#               return_tensors='tf')

# x_val= bert_tokenizer(X_valid[:num_val_examples], 
#               max_length=max_length,
#               truncation=True,
#               padding='max_length', 
#               return_tensors='tf')

x_train = bert_tokenizer(X_train, 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

x_test = bert_tokenizer(X_test, 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')

x_val= bert_tokenizer(X_valid, 
              max_length=max_length,
              truncation=True,
              padding='max_length', 
              return_tensors='tf')





def select_min_length_examples(x_data, y_data):

  x_input_ids = []
  y_labels = []

  for ((input_ids, masks), label) in zip(zip(x_data['input_ids'], x_data['attention_mask']), y_data):
    if masks[-1] == 1:
      x_input_ids.append(input_ids)
      y_labels.append(label)

  return np.array(x_input_ids), np.array(y_labels) 


In [44]:
# y_train = y_train[:num_train_examples]
# y_test = y_train[:num_train_examples]
# y_val = y_train[:num_val_examples]
y_train = y_train
y_test = y_test
y_val = y_valid

In [45]:
x_train

{'input_ids': <tf.Tensor: shape=(63972, 50), dtype=int32, numpy=
array([[  101, 18012,   131, ...,     0,     0,     0],
       [  101,   138,  8661, ...,   121,   168,   102],
       [  101,   138,  8661, ...,  1955,  1830,   102],
       ...,
       [  101, 18012,   131, ...,     0,     0,     0],
       [  101,   138,  8661, ...,   119,   121,   102],
       [  101,   138,  8661, ...,  1955,  1830,   102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(63972, 50), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(63972, 50), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=in

In [46]:
x_val

{'input_ids': <tf.Tensor: shape=(7997, 50), dtype=int32, numpy=
array([[ 101,  138, 8661, ..., 1545, 3556,  102],
       [ 101,  138, 8661, ...,  121,  119,  102],
       [ 101,  138, 8661, ..., 1830, 2087,  102],
       ...,
       [ 101,  138, 8661, ...,  119,  121,  102],
       [ 101,  138, 8661, ..., 1830, 2087,  102],
       [ 101,  138, 8661, ...,  119,  121,  102]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(7997, 50), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(7997, 50), dtype=int32, numpy=
array([[1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 1, 1, 1]], dtype=int32)>}

In [47]:
bert_train_input_ids, bert_train_labels = select_min_length_examples(x_train, y_train)
bert_test_input_ids, bert_test_labels = select_min_length_examples(x_test, y_test)
bert_val_input_ids, bert_val_labels = select_min_length_examples(x_val, y_val)

In [48]:
SEQUENCE_LENGTH = 50

In [49]:
def create_bert_cls_model(hidden_size = 100, 
                          dropout=0.3,
                          learning_rate=0.0005):
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """


    

    ### YOUR CODE HERE
    input_ids = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    #token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    #attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    
    bert_inputs = {'input_ids': input_ids}  

 

    bert_out = bert_model(bert_inputs)

    pooled_token = bert_out[1]
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooled_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')


    ### END YOUR CODE
    print(classification_model.summary())
    return classification_model

In [50]:
bert_classification_model = create_bert_cls_model()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids_layer (InputLayer  [(None, 50)]             0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             50, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                    

In [51]:
b_model = create_bert_cls_model()
history_wan = b_model.fit(bert_train_input_ids,
                            bert_train_labels,
                            validation_data=(bert_val_input_ids,bert_val_labels),
                            batch_size=8,
                            epochs=2
                            )

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids_layer (InputLayer  [(None, 50)]             0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             50, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                    

In [52]:
conf_matrix(b_model,bert_val_input_ids,bert_val_labels)

Precision: 100.0
Recall: 99.90426041168023
F1 Score: 99.9521072796935


(99.90426041168023, 99.9521072796935, array([[6261,    0],
        [   6,    0]]))

In [53]:
def create_bert_cls_model1(hidden_size = 100, 
                          dropout=0.3,
                          learning_rate=0.0005):
    """
    Build a simple classification model with BERT. Use the CLS Token output for classification purposes.
    """


    

    ### YOUR CODE HERE
    input_ids = tf.keras.layers.Input(shape=(SEQUENCE_LENGTH,), dtype=tf.int64, name='input_ids_layer')
    #token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    #attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    
    bert_inputs = {'input_ids': input_ids}  

 

    bert_out = bert_model(bert_inputs)

    #pooled_token = bert_out[1]
    cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(cls_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')


    ### END YOUR CODE
    print(classification_model.summary())
    return classification_model

In [54]:
bert_classification_model1 = create_bert_cls_model1()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids_layer (InputLayer  [(None, 50)]             0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             50, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                    

In [55]:
b_model1 = create_bert_cls_model1()
history_wan1 = b_model1.fit(bert_train_input_ids,
                            bert_train_labels,
                            validation_data=(bert_val_input_ids,bert_val_labels),
                            batch_size=8,
                            epochs=2
                            )

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids_layer (InputLayer  [(None, 50)]             0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             50, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                    

6266/6266 [==============================] - 829s 130ms/step - loss: 0.0098 - accuracy: 0.9989 - val_loss: 0.0106 - val_accuracy: 0.9990
Epoch 2/2
6266/6266 [==============================] - 808s 129ms/step - loss: 0.0092 - accuracy: 0.9992 - val_loss: 0.0108 - val_accuracy: 0.9990


In [56]:
conf_matrix(b_model1,bert_val_input_ids,bert_val_labels)

Precision: 100.0
Recall: 99.90426041168023
F1 Score: 99.9521072796935


(99.90426041168023, 99.9521072796935, array([[6261,    0],
        [   6,    0]]))

In [57]:
def create_bert_classification_model(bert_model,
                                     train_layers=-1,
                                     hidden_size = 200, 
                                     dropout=0.3,
                                     learning_rate=0.001):
    """
    Build a simple classification model with BERT. Use the Pooled Output for classification purposes
    """
    if train_layers == -1:
        # Freeze all layers of pre-trained BERT model
        bert_model.trainable = False

    else:
        # Restrict training to the train_layers outer transformer layers
        retrain_layers = []

        for retrain_layer_number in range(train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)
          
        
        print('retrain layers: ', retrain_layers)

        for w in bert_model.weights:
            if not any([x in w.name for x in retrain_layers]):
                print('freezing: ', w)
                w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}      

    bert_out = bert_model(bert_inputs)

    pooled_token = bert_out[1]
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooled_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')
    
    return classification_model

In [58]:
bert_classification_model = create_bert_classification_model(bert_model, train_layers=-1)

In [59]:
bert_classification_model.summary()

Model: "model_6"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 attention_mask_layer (InputLay  [(None, 50)]        0           []                               
 er)                                                                                              
                                                                                                  
 input_ids_layer (InputLayer)   [(None, 50)]         0           []                               
                                                                                                  
 token_type_ids_layer (InputLay  [(None, 50)]        0           []                               
 er)                                                                                              
                                                                                            

In [60]:
xtrain_id=np.array(x_train.input_ids)
xtrain_token=np.array(x_train.token_type_ids)
xtrain_attn=np.array(x_train.attention_mask)
y_train=np.array(y_train)
xtest_id=np.array(x_test.input_ids)
xtest_token=np.array(x_test.token_type_ids)
xtest_attn=np.array(x_test.attention_mask)
y_test=np.array(y_test)

In [61]:
bert_classification_model_history = bert_classification_model.fit(
    [xtrain_id, xtrain_token, xtrain_attn],
    y_train,
    validation_data=([xtest_id, xtest_token, xtest_attn], y_test),
    batch_size=32,
    epochs=2
)  

Epoch 1/2
2000/2000 [==============================] - 289s 139ms/step - loss: 0.2644 - accuracy: 0.9302 - val_loss: 0.2546 - val_accuracy: 0.9308
Epoch 2/2
2000/2000 [==============================] - 274s 137ms/step - loss: 0.2602 - accuracy: 0.9305 - val_loss: 0.2515 - val_accuracy: 0.9308


In [62]:
conf_matrix(bert_classification_model,[xtest_id, xtest_token, xtest_attn],y_test)

Precision: 100.0
Recall: 93.08490684006503
F1 Score: 96.41862573667508


(93.08490684006503, 96.41862573667508, array([[7444,    0],
        [ 553,    0]]))

In [63]:
def create_bert_train(bert_model,
                                     train_layers=1,
                                     hidden_size = 200, 
                                     dropout=0.3,
                                     learning_rate=0.001):
    """
    Build a simple classification model with BERT. Use the Pooled Output for classification purposes
    """
    if train_layers == -1:
        # Freeze all layers of pre-trained BERT model
        bert_model.trainable = False

    else:
        # Restrict training to the train_layers outer transformer layers
        retrain_layers = []

        for retrain_layer_number in range(train_layers):

            layer_code = '_' + str(11 - retrain_layer_number)
            retrain_layers.append(layer_code)
          
        
        print('retrain layers: ', retrain_layers)

        for w in bert_model.weights:
            if not any([x in w.name for x in retrain_layers]):
                print('freezing: ', w)
                w._trainable = False

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}      

    bert_out = bert_model(bert_inputs)

    pooled_token = bert_out[1]
    #cls_token = bert_out[0][:, 0, :]

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(pooled_token)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  


    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')
    
    return classification_model

In [64]:
bert_classification_model_1 = create_bert_train(bert_model, train_layers=1)

Streaming output truncated to the last 5000 lines.
       -3.84457372e-02,  1.42489420e-02,  8.66156351e-03, -6.66948706e-02,
        2.57130642e-03,  2.77135037e-02,  7.06250444e-02,  1.69825610e-02,
        8.02297443e-02,  1.75964972e-03,  9.41159576e-03, -7.26500526e-02,
        1.35728782e-02,  4.77390289e-02,  8.55601300e-03, -6.97698593e-02,
       -5.47264889e-02,  8.71734868e-04, -3.68627198e-02, -1.81683842e-02,
       -1.02772214e-01,  6.55520186e-02, -2.00032592e-02, -3.11313607e-02,
        6.21170597e-03,  1.64006203e-02, -1.63658604e-01, -2.31642947e-02,
        1.03909464e-03, -8.06150734e-02, -1.41791962e-02,  3.65319243e-03,
        7.10767955e-02,  4.64817695e-02, -4.44593988e-02,  3.94027121e-02,
       -3.85911986e-02,  7.96867684e-02,  1.08380597e-02,  5.94226352e-04,
       -4.45972271e-02, -9.32011195e-03,  3.83806452e-02, -3.19965594e-02,
        5.15427962e-02, -3.28382081e-03,  4.15987708e-02, -1.40592381e-02,
       -6.28966019e-02,  6.59951270e-02, -1.08284

In [65]:
bert_classification_model_history1 = bert_classification_model_1.fit(
    [xtrain_id, xtrain_token, xtrain_attn],
    y_train,
    validation_data=([xtest_id, xtest_token, xtest_attn], y_test),
    batch_size=32,
    epochs=2
)  

Epoch 1/2
2000/2000 [==============================] - 282s 138ms/step - loss: 0.2683 - accuracy: 0.9303 - val_loss: 0.2556 - val_accuracy: 0.9308
Epoch 2/2
2000/2000 [==============================] - 273s 137ms/step - loss: 0.2624 - accuracy: 0.9305 - val_loss: 0.2516 - val_accuracy: 0.9308


In [66]:
conf_matrix(bert_classification_model,[xtest_id, xtest_token, xtest_attn],y_test)

Precision: 100.0
Recall: 93.08490684006503
F1 Score: 96.41862573667508


(93.08490684006503, 96.41862573667508, array([[7444,    0],
        [ 553,    0]]))

In [67]:
def create_bert_avg_model(hidden_size = 100, 
                                dropout=0.3,
                                learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the average of the BERT output tokens
    """

    ### YOUR CODE HERE
    ### YOUR CODE HERE
    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    #token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    #attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    
    bert_inputs = {'input_ids': input_ids}  

 

    bert_out = bert_model(bert_inputs)


    pooled_token = bert_out[0][:, 1:-1, :]
    #cls_token = bert_out[0][:, 0, :]
    bert_average=tf.math.reduce_mean(pooled_token,axis=1)

    #bert_average= tf.keras.layers.Reshape((bert_average.shape[-1],))

    hidden = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer')(bert_average)


    hidden = tf.keras.layers.Dropout(dropout)(hidden)  

    classification = tf.keras.layers.Dense(1, activation='sigmoid',name='classification_layer')(hidden)
    
    classification_model = tf.keras.Model(inputs=[input_ids], outputs=[classification])
    
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                                 loss=tf.keras.losses.BinaryCrossentropy(from_logits=False), 
                                 metrics='accuracy')


    ### END YOUR CODE
    print(classification_model.summary())
    return classification_model


In [68]:
b_model1 = create_bert_avg_model()
history_wan = b_model1.fit(bert_train_input_ids,
                            bert_train_labels,
                            validation_data=(bert_test_input_ids,bert_test_labels),
                            batch_size=8,
                            epochs=2
                            )

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_ids_layer (InputLayer  [(None, 50)]             0         
 )                                                               
                                                                 
 tf_bert_model (TFBertModel)  TFBaseModelOutputWithPoo  108310272
                             lingAndCrossAttentions(l            
                             ast_hidden_state=(None,             
                             50, 768),                           
                              pooler_output=(None, 76            
                             8),                                 
                              past_key_values=None, h            
                             idden_states=None, atten            
                             tions=None, cross_attent            
                             ions=None)                    

In [69]:
conf_matrix(b_model1,bert_test_input_ids,bert_test_labels)

Precision: 100.0
Recall: 99.95159728944822
F1 Score: 99.97579278625031


(99.95159728944822, 99.97579278625031, array([[6195,    0],
        [   3,    0]]))

In [70]:
def create_bert_cnn_model(hidden_size = 100, 
                                learning_rate=0.00005,
                                num_filters = [100, 100, 50, 25],
                                kernel_sizes = [3, 5, 10, 20],
                                dropout = 0.3):
    """
    Build a  classification model with BERT, where you apply CNN layers  to the BERT output
    """

    ### YOUR CODE HERE

    input_ids = tf.keras.layers.Input(shape=(50,), dtype=tf.int64, name='input_ids_layer')
    #token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    #attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids}  

    bert_out = bert_model(bert_inputs)
 


    #pooled_token = bert_out[1]
    #pooled_token = bert_out[0]
    #cls_token = bert_out[0][:, 0, :]
  #bert_average=tf.math.reduce_mean(pooled_token,axis=1)


    conv_layers_for_all_kernel_sizes = []
    for kernel_size, filters in zip(kernel_sizes, num_filters):
        conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(bert_out[0])
        conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
        conv_layers_for_all_kernel_sizes.append(conv_layer)
    
    conv_output = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1)
    #last_hidden_output = keras.layers.Dropout(rate=dropout)(conv_output)

    
    last_hidden_output = keras.layers.Dense(hidden_size, activation='relu')(conv_output)
    last_hidden_output = keras.layers.Dropout(dropout)(last_hidden_output)

    #flatt=keras.layers.Flatten()(last_hidden_output)
    #cnn_prediction = keras.layers.Dense(1, activation='sigmoid')(last_hidden_output)
    cnn_prediction = keras.layers.Dense(1, activation='sigmoid')(last_hidden_output)

    classification_model = keras.Model(inputs=input_ids, outputs=cnn_prediction)
    classification_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
                      loss='binary_crossentropy',  # From information theory notebooks.
                      metrics=['accuracy'])
   
    ### END YOUR CODE
    
    print(classification_model.summary())
    return classification_model

In [71]:
cnn_bert_model=create_bert_cnn_model()
cnn_bert_model.summary

Model: "model_9"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids_layer (InputLayer)   [(None, 50)]         0           []                               
                                                                                                  
 tf_bert_model (TFBertModel)    TFBaseModelOutputWi  108310272   ['input_ids_layer[0][0]']        
                                thPoolingAndCrossAt                                               
                                tentions(last_hidde                                               
                                n_state=(None, 50,                                                
                                768),                                                             
                                 pooler_output=(Non                                         

<bound method Model.summary of <keras.engine.functional.Functional object at 0x7f62b922e370>>

In [72]:
cnn_bert_history = cnn_bert_model.fit(bert_train_input_ids,
                            bert_train_labels,
                            validation_data=(bert_test_input_ids,bert_test_labels),
                            batch_size=8,
                            epochs=2
                            )

Epoch 1/2
6266/6266 [==============================] - 364s 56ms/step - loss: 0.0084 - accuracy: 0.9991 - val_loss: 0.0048 - val_accuracy: 0.9995
Epoch 2/2
6266/6266 [==============================] - 345s 55ms/step - loss: 0.0080 - accuracy: 0.9992 - val_loss: 0.0053 - val_accuracy: 0.9995


In [73]:
conf_matrix(cnn_bert_model,bert_test_input_ids,bert_test_labels)

Precision: 100.0
Recall: 99.95159728944822
F1 Score: 99.97579278625031


(99.95159728944822, 99.97579278625031, array([[6195,    0],
        [   3,    0]]))